In [ ]:
! pip install streamlit -q
! pip install datasets transformers trl peft accelerate bitsandbytes optimum
! git lfs install

!pip uninstall -y bitsandbytes==0.41.1
!pip uninstall -y torch torchvision torchaudio
!pip uninstall -y auto-gptq
!pip install torch torchvision torchaudio --trusted-host download.pytorch.org --index-url https://download.pytorch.org/whl/cu118 --no-cache-dir
!pip install bitsandbytes --no-cache-dir
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/


# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install streamlit_option_menu

!git clone https://huggingface.co/Aneeth/zephyr_7k
# !git clone https://huggingface.co/BevenRozario/mistral_v3_5k


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
%%writefile app.py
import streamlit as st
import requests
from streamlit_option_menu import option_menu
import base64

import streamlit as st
import torch
import requests
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig, AutoTokenizer
from streamlit_option_menu import option_menu

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel





def home_page():
    # page_bg_img = '''
    # <style>
    # body {
    # background-image: url("");
    # background-size: cover;
    # }
    # </style>
    # '''
    # st.markdown(page_bg_img, unsafe_allow_html=True)
    # @st.cache_resource(allow_output_mutation=True)
    def add_bg_from_local(image_file):
        with open(image_file, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())
        st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url(data:image/{"png"};base64,{encoded_string.decode()});
            background-size: cover
        }}
        </style>
        """,
        unsafe_allow_html=True
        )
    add_bg_from_local('/content/drive/MyDrive/women5.png')

    st.markdown("<h1 style='text-align: center; font-size: 4.5em; color: #DE4907;'>Still Handcrafting Job Descriptions..?</h1>", unsafe_allow_html=True)
    st.markdown("<h5 style='text-align: center; font-size: 2.5em; color: #DE4907;'><em>Let AI streamline your hiring process by saving 75% of the time it takes to write a Job Description!</em></h5>", unsafe_allow_html=True)
def model_zephyr(job_title, job_skills):
    #Define job title and skills
    Job_title = job_title.lower()
    job_skills = job_skills.lower()

    # Process data sample
    def process_data_sample(example):
        processed_example = "\n Generate an authentic job description using the given input.\n\n" + example["instruction"] + "\n\n"
        return processed_example

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained("/content/zephyr_7k")

    # Prepare input string
    inp_str = process_data_sample(
    {
        "instruction": f"Generate a job description for a {Job_title} role. The job description should only include following skills: {job_skills}. Not any other skills!",
    }
    )

    # Tokenize inputs
    inputs = tokenizer(inp_str, return_tensors="pt").to("cuda")

    # Load model
    model = AutoPeftModelForCausalLM.from_pretrained(
        "/content/zephyr_7k",
        low_cpu_mem_usage=True,
        return_dict=True,
        torch_dtype=torch.float16,
        device_map="cuda")

    # Generation config
    generation_config = GenerationConfig(
        do_sample=True,
        top_k=1,
        temperature=0.8,
        max_new_tokens=5000,
        pad_token_id=tokenizer.eos_token_id)

    # Generate job description
    outputs = model.generate(**inputs, generation_config=generation_config)
    generated_description = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_description


def explore_sidebar():
    st.sidebar.title("Explore Sidebar")
    st.sidebar.write("This is the sidebar content.")
    # Add more content specific to the sidebar if needed.
def write_jd_page():
    st.title("Ready to Write JDs effortlessly?")

    # Create two columns
    left_column, right_column = st.columns(2)

    # Left column for user inputs
    with left_column:
        job_role = st.text_input("What is the Job Role ?", key="job_role_input")
        skills = st.text_input("Is there any specific Skills ?", key="skills_input")

        generate_button_enabled = bool(job_role)
        generate_button = st.button("Generate JD", key="generate_button", disabled=not generate_button_enabled)

    # Right column for generated text
    with right_column:
        if generate_button:
            generated_description_zephyr = model_zephyr(job_role, skills)
            # Find the index of "Job Description:"
            index = generated_description_zephyr.find("Job Description:")
            # Extract the substring starting from "Job Description:"
            job_description = generated_description_zephyr[index:]
            st.subheader("Generated Job Description")
            st.markdown(job_description)


def survey_page():
    st.cache_data.clear()
    st.cache_resource.clear()
    st.title("Which JD would you prefer?")
    st.write("Feel free to rate model based on the given Criteria")

    left_column, right_column = st.columns(2)

    # Left column for user inputs
    with left_column:

        text_input_job_role = st.text_input(
            "What is the Job Role?",
            value="",
            max_chars=100,
            key="input_job_title",
            help="Enter the job title.",
            # style="width: 150px;",
        )

        text_input_skills = st.text_input(
            "Are there any specific skills?",
            value="",
            max_chars=100,
            key="input_job_skills",
            help="Enter specific skills if any.",
            # style="width: 150px;",
        )
    # Generate button
    if st.button("Generate"):
        if text_input_job_role.strip() == "" or text_input_skills.strip() == "":
            st.warning("Please fill in all fields.")
        else:

            col1, col2 = st.columns(2)
            generated_description_zephyr = model_zephyr(text_input_job_role, text_input_skills)
            # Find the index of "Job Description:"
            index = generated_description_zephyr.find("Job Description:")

            # Extract the substring starting from "Job Description:"
            job_description = generated_description_zephyr[index:]

            with col1:
              st.subheader("Model A")
              st.markdown(job_description)


            api_url = 'https://c587-35-204-90-203.ngrok-free.app//run_mistral'
            params = {
                'job_role': text_input_job_role,
                'job_skills': text_input_skills
            }
            response = requests.get(api_url, params=params)


            with col2:
              st.subheader("Model B")
              st.markdown(str(response.text))


            criteria = ['Correctness', 'Relevancy', 'Conciseness', 'Inclusivity']
            models = ['Model A', 'Model B', 'Both', 'None']

            st.markdown(
                """<iframe src="https://docs.google.com/forms/d/e/1FAIpQLSfhIU2381rzHKKQzrMSqmn5Yr2GhgxsfG0khcv74a5nHZqloQ/viewform?embedded=true" width="100%" height="2500" style="border:none; overflow:hidden;" scrolling="no" frameborder="0" allowTransparency="true" allowfullscreen="true">Loading...</iframe>""",
                unsafe_allow_html=True,
            )




    # Create Streamlit UI
    # st.write("<p style='font-size:22px;'>Tell us Your Prefernce by Rating the Models based on the given criteria.</p>", unsafe_allow_html=True)


    # # Create the first row with headers
    # header_columns = st.columns(5)
    # with header_columns[0]:
    #     st.write('Criteria')
    # for model in models:
    #     with header_columns[models.index(model) + 1]:
    #         st.write(model)

    # # Create the checkboxes
    # for i, criterion in enumerate(criteria):
    #     row_columns = st.columns(5)
    #     with row_columns[0]:
    #         st.write(criterion)
    #     for j, model in enumerate(models):
    #         with row_columns[j + 1]:
    #             st.checkbox("", key=f"{i}-{j}")

    # # Add a button to submit feedback
    # if st.button("Submit Feedback"):
    #     # Process the selected checkboxes
    #     selected_criteria = [criteria[i] for i in range(4) if st.checkbox("", key=f"{i}-0")]
    #     selected_models = [models[j - 1] for j in range(1, 5) if st.checkbox("", key=f"0-{j}")]
    #     st.success(f"Feedback submitted! Criteria: {', '.join(selected_criteria)}, Models: {', '.join(selected_models)}")



def main():

    top_left_logo_path = "/content/drive/MyDrive/logo.svg"
    st.set_page_config(
        page_title="Job Description Generator",
        layout="wide",  # or "full"
        initial_sidebar_state="expanded",
        page_icon=":rocket:",  # optional icon
        menu_items={
            "Get Help": "https://github.com/streamlit/streamlit",
            "Report a Bug": "https://github.com/streamlit/streamlit/issues",
            "About": "# This is a Streamlit app for your job description project.",
        },
    )



    # st.image(top_left_logo_path, width=550)
    def streamlit_menu(example=1):
        if example == 1:
            # 1. as sidebar menu
            with st.sidebar:
                selected = option_menu(
                    menu_title= None,  # required
                    options=["Home", "Write a JD", "Rate"],  # required
                    icons=["house", "book", "envelope"],  # optional
                    menu_icon="cast",  # optional
                    default_index=0,  # optional
                )
            return selected

        if example == 2:
            # 2. horizontal menu w/o custom style
            selected = option_menu(
                menu_title=None,  # required
                options=["Home", "Write a JD", "Tell us your Preference"],  # required
                icons=["house", "pen", "stars"],  # optional
                menu_icon="cast",  # optional
                default_index=0,  # optional
                orientation="horizontal",
            )
            return selected
    # st.markdown(
    #     """
    #     <style>
    #         .element-container:nth-child(1) {
    #             flex: 0 1 10px;  /* Set the minimum width of the left column */
    #             flex-shrink: 5;
    #         }
    #         .element-container:nth-child(2) {
    #             flex-grow: 1;  /* Allow the right column to grow and take available space */
    #         }
    #     </style>
    #     """,
    #     unsafe_allow_html=True,
    # )
    left_column, right_column = st.columns(2)



    # Left column for user inputs
    with left_column:
        st.image(top_left_logo_path, width=250)
    with right_column:
        selected = streamlit_menu(example=2)

    if selected == "Home":
        home_page()
    elif selected == "Write a JD":
        write_jd_page()
    elif selected == "Tell us your Preference":
        survey_page()

if __name__ == "__main__":
    main()



Overwriting app.py


In [ ]:
!pip install pyngrok==7.0.5
# !pip install pyngrok
import pyngrok

In [ ]:

from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2cVf1Omx7iJVbPUVzVgLjDBFHGy_5RKCjurFsA1gjgPZQQwQz") #ngrok.com
#!ngrok config add-authtoken 2cEfn0CnLAhac1eoNpSQfqLihD9_7g2btyTRmMivd2q7PKDR7


In [ ]:
!nohup streamlit run app.py --server.port 80 &

ngrok.connect(80)

nohup: appending output to 'nohup.out'


<NgrokTunnel: "https://dbfd-34-168-173-30.ngrok-free.app" -> "http://localhost:80">